In [1]:
import os
from skimage import io, transform
import numpy as np
from tqdm import tqdm
from model import FireNet

import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

In [2]:
net = FireNet()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device)

FireNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (drop1): Dropout2d(p=0.5, inplace=False)
  (drop2): Dropout(p=0.2, inplace=False)
  (dense1): Linear(in_features=2304, out_features=256, bias=True)
  (dense2): Linear(in_features=256, out_features=128, bias=True)
  (dense3): Linear(in_features=128, out_features=2, bias=True)
)

In [3]:
TRAINING_PATH='Dataset'
CATEGORIES = ['Fire', 'NoFire']

In [4]:
class Rescale(object):
    def __init__(self, output_size):
        assert isinstance(output_size, tuple)
        self.output_size = output_size
        
    def __call__(self, sample):
        image, class_num = sample['image'], sample['class']
        h, w = image.shape[:2]
        new_h, new_w = self.output_size
        new_h, new_w = int(new_h), int(new_w)
        img = transform.resize(image, (new_h, new_w))
        return {'image': img, 'class': class_num}
    
class ToTensor(object):
    def __call__(self, sample):
        image, class_num = sample['image'], sample['class']
        image = image.transpose((2, 0, 1))
        return {'image': torch.from_numpy(image), 'class': class_num}

In [5]:
class TrainingSet(Dataset):
    def __init__(self, transform=None):
        self.transform = transform
        classes = []
        images = []
        for category in CATEGORIES:
            path = os.path.join(TRAINING_PATH, category)
            class_num = CATEGORIES.index(category)
            
            for img in tqdm(os.listdir(path)):
                try:
                    image = io.imread(os.path.join(path, img))
                    if (image.shape[2] == 3):
                        images.append(image)
                        classes.append(class_num)
                except Exception as e:
                    pass
        self.set = {'image': images, 'class': classes}
    def __len__(self):
        return len(self.set['class'])
    def __getitem__(self, idx):
        image = self.set['image'][idx]
        classe = self.set['class'][idx]
        sample = {'image': image, 'class': classe}
        if self.transform:
            composed = transforms.Compose([Rescale((64, 64)), ToTensor()])
            sample = composed(sample)
        return sample
            
        

In [6]:
training_set= TrainingSet(transform=True)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1280/1280 [00:41<00:00, 30.80it/s]


In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

In [9]:
trainloader = DataLoader(training_set, batch_size=32, shuffle=True, num_workers=0)
net = net.float()

In [10]:
for epoch in range(2):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data['image'].to(device), data['class'].to(device)

        optimizer.zero_grad()

        outputs = net(inputs.float())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 10 == 9:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i+1, running_loss/10))
            running_loss = 0.0

[1,    10] loss: 0.682
[1,    20] loss: 0.679
[1,    30] loss: 0.681
[1,    40] loss: 0.645
[1,    50] loss: 0.610
[1,    60] loss: 0.639
[1,    70] loss: 0.648
[2,    10] loss: 0.651
[2,    20] loss: 0.629
[2,    30] loss: 0.629
[2,    40] loss: 0.572
[2,    50] loss: 0.650
[2,    60] loss: 0.600
[2,    70] loss: 0.597


In [11]:
torch.save(net.state_dict(), './trained_weights.pth')